# Model effectivness Verification (single sample)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch_geometric.nn import GCNConv
from customize_set import SyntheticSet_2

dataset = SyntheticSet_2('./data/synthetic_2')


class GCN(nn.Module):
    def __init__(self, hidden_channels) -> None:
        super().__init__()
        torch.manual_seed(2023)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(hidden_channels,256)
        self.fc2 = torch.nn.Linear(256,1)
       
    
    def forward(self, x, edge_index):
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        # x = F.softmax(x,dim=1)

        return x


Processing...
Done!


In [2]:
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.data import DataLoader

writer = SummaryWriter()


EPOCH = 10
LR = 0.01
train_ratio = 0.8

# setting device
if torch.backends.mps.is_available:
    device = torch.device('mps')
elif torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


train_len = int(len(dataset)*0.8)
train_set = DataLoader(dataset[:train_len])
test_set = DataLoader(dataset[train_len:])

model = GCN(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=5e-4)
# rank_loss = RankLoss('./data', device)
mse_loss = nn.MSELoss()

# record the network structure
dummy_sample = dataset[0].to(device)
writer.add_graph(model, [dummy_sample.x, dummy_sample.edge_index] )


# training

for epoch in range(EPOCH):
    model.train()
    train_loss = 0
    for sample in train_set:
        optimizer.zero_grad()
        out = model(sample.x, sample.edge_index)
        loss = mse_loss(out, sample.y)
        loss.backward()  
        optimizer.step()  
        train_loss += loss
    
    model.eval()
    test_loss = 0
    for sample in test_set:
        out = model(sample.x, sample.edge_index)
        loss = mse_loss(out, sample.y)
        test_loss += loss

    print("-Epoch: {} | Training Loss: {} | Testing Loss: {}".format(epoch, train_loss, test_loss))
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/test', test_loss, epoch)


writer.close()


/opt/anaconda3/envs/GNN/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


RuntimeError: index 108 is out of bounds for dimension 0 with size 108